In [1]:
import findspark, datetime, pytz
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import lit,row_number,col,count

findspark.add_packages('mysql:mysql-connector-java:8.0.11')

In [2]:
agora = datetime.datetime.now(pytz.timezone('America/Sao_paulo'))
strBronzePath = f'./LAKE/BRONZE/loja_simulada-vendas'

In [15]:
# Create SparkSession
spark = (SparkSession.builder
           .appName('IngestaoBronze')
           .config("packages", "org.apache.spark:mysql-connector-java-8.0.13.jar")
           .getOrCreate()
        )

In [4]:
def pegarBronzeAtual(tabela):
    try:
        tempDF = (spark.read.parquet(f'{strBronzePath}-{tabela}')).cache()
        print(f'Base BRONZE "{tabela}" encontrada!')
        return tempDF
    except Exception as e:
        print(f'Base BRONZE "{tabela}" NÃO encontrada!\n', e)
        return None

In [5]:
# Read from MySQL Table
def pegarDadosDaTable(tabela):
    return (spark.read
            .format("jdbc")
            .option("driver","com.mysql.cj.jdbc.Driver")
            .option("url", "jdbc:mysql://localhost:3306/vendas?allowPublicKeyRetrieval=true&useSSL=false")
            .option("dbtable", tabela)
            .option("user", "big_data_importer")
            .option("password", "big_data_importer")
            .load()
         )

In [6]:
def enviarAlerta(tipoAlerta, mensagem):
    print(F'Encaminhando alerta de {tipoAlerta}')
    print(f'{tipoAlerta} - type: {type(mensagem)} - Mensagem:', mensagem)
    return

In [7]:
def salvarFinalDF(df,tabela):
    return df.write.mode("overwrite").parquet(f'{strBronzePath}-{tabela}')

In [8]:
def clientes():
    tabela = 'clientes'
    bronzeAtualDF = pegarBronzeAtual(tabela)
    baseNovaDF = pegarDadosDaTable(tabela).withColumn("data_carga", lit(agora.strftime("%Y-%m-%d")).cast('date'))
    enviarAlerta('Informacao', f'COUNT bronze {tabela}: {baseNovaDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    if bronzeAtualDF != None:
   # SELECIONAR DADOS NOVOS
        apenasNovosDF = (baseNovaDF.join(bronzeAtualDF          
            ,((bronzeAtualDF.id_cliente == baseNovaDF.id_cliente) &
                (bronzeAtualDF.cpf == baseNovaDF.cpf) &
                (bronzeAtualDF.telefone == baseNovaDF.telefone) &
                (bronzeAtualDF.email == baseNovaDF.email) &
                (bronzeAtualDF.cliente == baseNovaDF.cliente) &
                (bronzeAtualDF.estado == baseNovaDF.estado) &
                (bronzeAtualDF.origem_racial == baseNovaDF.origem_racial) &
                (bronzeAtualDF.sexo == baseNovaDF.sexo) &
                (bronzeAtualDF.status == baseNovaDF.status))
            ,"leftanti"))
        enviarAlerta('Informacao', f'COUNT novos registros: {apenasNovosDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    # ENCONTRAR ULTIMA SURROGATE KEY
        maxSK = bronzeAtualDF.selectExpr("max(surrogate_key) as max_key").collect()[0]["max_key"]
    # ADICIONAR SK AO NOVO DF
        windowSpec = Window.orderBy("id_cliente")        
        apenasNovosSKDF = (apenasNovosDF.withColumn("surrogate_key", maxSK + row_number().over(windowSpec)))
    # FINAL DO PROCESSO
        finalDF = bronzeAtualDF.unionByName(apenasNovosSKDF)
        salvarFinalDF(finalDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {apenasNovosDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    else:
        windowSpec = Window.orderBy("id_cliente")
        bronzeNovoDF = baseNovaDF.withColumn("surrogate_key", row_number().over(windowSpec))
        salvarFinalDF(bronzeNovoDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {bronzeNovoDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    return

In [9]:
def vendedores():
    tabela = 'vendedores'
    bronzeAtualDF = pegarBronzeAtual(tabela)
    baseNovaDF = pegarDadosDaTable(tabela).withColumn("data_carga", lit(agora.strftime("%Y-%m-%d")).cast('date'))
    enviarAlerta('Informacao', f'COUNT bronze {tabela}: {baseNovaDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    if bronzeAtualDF != None:
    # SELECIONAR DADOS NOVOS
        apenasNovosDF = (baseNovaDF.join(bronzeAtualDF          
            ,((bronzeAtualDF.id_vendedor == baseNovaDF.id_vendedor) &
                (bronzeAtualDF.cpf == baseNovaDF.cpf) &
                (bronzeAtualDF.telefone == baseNovaDF.telefone) &
                (bronzeAtualDF.email == baseNovaDF.email) &
                (bronzeAtualDF.origem_racial == baseNovaDF.origem_racial) &
                (bronzeAtualDF.nome == baseNovaDF.nome) )
            ,"leftanti"))
        enviarAlerta('Informacao', f'COUNT novos registros: {apenasNovosDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    # ENCONTRAR ULTIMA SURROGATE KEY
        maxSK = bronzeAtualDF.selectExpr("max(surrogate_key) as max_key").collect()[0]["max_key"]
    # ADICIONAR SK AO NOVO DF
        windowSpec = Window.orderBy("id_vendedor")        
        apenasNovosSKDF = (apenasNovosDF.withColumn("surrogate_key", maxSK + row_number().over(windowSpec)))
    # FINAL DO PROCESSO
        finalDF = bronzeAtualDF.unionByName(apenasNovosSKDF)
        salvarFinalDF(finalDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {finalDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    else:
        windowSpec = Window.orderBy("id_vendedor")
        bronzeNovoDF = baseNovaDF.withColumn("surrogate_key", row_number().over(windowSpec))
        salvarFinalDF(bronzeNovoDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {bronzeNovoDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    return

In [10]:
def produtos():
    tabela = 'produtos'
    bronzeAtualDF = pegarBronzeAtual(tabela)
    baseNovaDF = pegarDadosDaTable(tabela).withColumn("data_carga", lit(agora.strftime("%Y-%m-%d")).cast('date'))
    enviarAlerta('Informacao', f'COUNT bronze {tabela}: {baseNovaDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    if bronzeAtualDF != None:
    # SELECIONAR DADOS NOVOS
        apenasNovosDF = (baseNovaDF.join(bronzeAtualDF          
            ,((bronzeAtualDF.id_produto == baseNovaDF.id_produto) &
                (bronzeAtualDF.produto == baseNovaDF.produto) &
                (bronzeAtualDF.preco == baseNovaDF.preco))
            ,"leftanti"))
        enviarAlerta('Informacao', f'COUNT novos registros: {apenasNovosDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    # ENCONTRAR ULTIMA SURROGATE KEY
        maxSK = bronzeAtualDF.selectExpr("max(surrogate_key) as max_key").collect()[0]["max_key"]
    # ADICIONAR SK AO NOVO DF
        windowSpec = Window.orderBy("id_produto")        
        apenasNovosSKDF = (apenasNovosDF.withColumn("surrogate_key", maxSK + row_number().over(windowSpec)))
    # FINAL DO PROCESSO
        finalDF = bronzeAtualDF.unionByName(apenasNovosSKDF)
        salvarFinalDF(finalDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {finalDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    else:
        windowSpec = Window.orderBy("id_produto")
        bronzeNovoDF = baseNovaDF.withColumn("surrogate_key", row_number().over(windowSpec))
        salvarFinalDF(bronzeNovoDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {bronzeNovoDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    return

In [11]:
def vendas():
    tabela = 'vendas'
    bronzeAtualDF = pegarBronzeAtual(tabela)
    baseNovaDF = pegarDadosDaTable(tabela).withColumn("data_carga", lit(agora.strftime("%Y-%m-%d")).cast('date'))
    enviarAlerta('Informacao', f'COUNT bronze {tabela}: {baseNovaDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    if bronzeAtualDF != None:
    # SELECIONAR DADOS NOVOS
        apenasNovosDF = (baseNovaDF.join(bronzeAtualDF          
            ,((bronzeAtualDF.id_venda == baseNovaDF.id_venda) &
                (bronzeAtualDF.id_vendedor == baseNovaDF.id_vendedor) &
                (bronzeAtualDF.id_cliente == baseNovaDF.id_cliente) &
                (bronzeAtualDF.data == baseNovaDF.data)) &
                (bronzeAtualDF.total == baseNovaDF.total)
            ,"leftanti"))
        enviarAlerta('Informacao', f'COUNT novos registros: {apenasNovosDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    # ENCONTRAR ULTIMA SURROGATE KEY
        maxSK = bronzeAtualDF.selectExpr("max(surrogate_key) as max_key").collect()[0]["max_key"]
    # ADICIONAR SK AO NOVO DF
        windowSpec = Window.orderBy("id_venda")        
        apenasNovosSKDF = (apenasNovosDF.withColumn("surrogate_key", maxSK + row_number().over(windowSpec)))
    # FINAL DO PROCESSO
        finalDF = bronzeAtualDF.unionByName(apenasNovosSKDF)
        salvarFinalDF(finalDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {finalDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    else:
        windowSpec = Window.orderBy("id_venda")
        bronzeNovoDF = baseNovaDF.withColumn("surrogate_key", row_number().over(windowSpec))
        salvarFinalDF(bronzeNovoDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {bronzeNovoDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    return

In [12]:
def itens_venda():
    tabela = 'itens_venda'
    bronzeAtualDF = pegarBronzeAtual(tabela)
    baseNovaDF = pegarDadosDaTable(tabela).withColumn("data_carga", lit(agora.strftime("%Y-%m-%d")).cast('date'))
    enviarAlerta('Informacao', f'COUNT bronze {tabela}: {baseNovaDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    if bronzeAtualDF != None:
    # SELECIONAR DADOS NOVOS
        # bronzeAtualDF = bronzeAtualDF1  # .select('*').where(col('id_venda') < 10)  # TODO inserida para teste, retirar
        apenasNovosDF = (baseNovaDF.join(bronzeAtualDF          
            ,((bronzeAtualDF.id_produto == baseNovaDF.id_produto) &
                (bronzeAtualDF.id_venda == baseNovaDF.id_venda) &
                (bronzeAtualDF.quantidade == baseNovaDF.quantidade) &
                (bronzeAtualDF.valor_unitario == baseNovaDF.valor_unitario)) &
                (bronzeAtualDF.valor_total == baseNovaDF.valor_total) &
                (bronzeAtualDF.desconto == baseNovaDF.desconto)
            ,"leftanti"))
        enviarAlerta('Informacao', f'COUNT novos registros: {apenasNovosDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    # ENCONTRAR ULTIMA SURROGATE KEY
        maxSK = bronzeAtualDF.selectExpr("max(surrogate_key) as max_key").collect()[0]["max_key"]
    # ADICIONAR SK AO NOVO DF
        windowSpec = Window.orderBy(*["id_venda", "id_produto"])
        apenasNovosSKDF = (apenasNovosDF.withColumn("surrogate_key", maxSK + row_number().over(windowSpec)))
    # FINAL DO PROCESSO
        finalDF = bronzeAtualDF.unionByName(apenasNovosSKDF)
        salvarFinalDF(finalDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {finalDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    else:
        windowSpec = Window.orderBy("id_venda")
        bronzeNovoDF = baseNovaDF.withColumn("surrogate_key", row_number().over(windowSpec))
        salvarFinalDF(bronzeNovoDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {bronzeNovoDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    return

In [13]:
def main():
    try:
        enviarAlerta('Informacao', 'ingestão para base BRONZE iniciada...')
        clientes()
        vendedores()
        produtos()
        vendas()
        itens_venda()
        enviarAlerta('Sucesso', 'Ingestão para base BRONZE concluída!!!')
    except Exception as e:
        enviarAlerta('Erro', e)
    finally:
        spark.stop()
        print('Conexão spark finalizada!')

In [14]:
main()

Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: ingestão para base BRONZE iniciada...
Base BRONZE "clientes" NÃO encontrada!
 [PATH_NOT_FOUND] Path does not exist: file:/home/cj/AULAS/LAKE/BRONZE/loja_simulada-vendas-clientes.
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: COUNT bronze clientes: 530


24/06/09 10:51:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 10:51:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 10:51:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 10:51:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 10:51:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: Registros a serem salvos: 530
Base BRONZE "vendedores" NÃO encontrada!
 [PATH_NOT_FOUND] Path does not exist: file:/home/cj/AULAS/LAKE/BRONZE/loja_simulada-vendas-vendedores.
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: COUNT bronze vendedores: 36


24/06/09 10:51:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 10:51:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 10:51:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 10:51:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 10:51:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: Registros a serem salvos: 36
Base BRONZE "produtos" NÃO encontrada!
 [PATH_NOT_FOUND] Path does not exist: file:/home/cj/AULAS/LAKE/BRONZE/loja_simulada-vendas-produtos.
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: COUNT bronze produtos: 14


...

Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: Registros a serem salvos: 14
Base BRONZE "vendas" NÃO encontrada!
 [PATH_NOT_FOUND] Path does not exist: file:/home/cj/AULAS/LAKE/BRONZE/loja_simulada-vendas-vendas.
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: COUNT bronze vendas: 73136


24/06/09 10:51:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 10:51:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 10:51:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 10:51:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 10:51:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: Registros a serem salvos: 73136
Base BRONZE "itens_venda" NÃO encontrada!
 [PATH_NOT_FOUND] Path does not exist: file:/home/cj/AULAS/LAKE/BRONZE/loja_simulada-vendas-itens_venda.
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: COUNT bronze itens_venda: 72633


24/06/09 10:51:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 10:51:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 10:51:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 10:51:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 10:51:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: Registros a serem salvos: 72633
Encaminhando alerta de Sucesso
Sucesso - type: <class 'str'> - Mensagem: Ingestão para base BRONZE concluída!!!
Conexão spark finalizada!
